
## NPC Hunter

We show how to use EZKL to prove that the NPC hunter updates are "optimal". Given coordinates x,y for the hunter and the prey, the hunter moves to the closest point in the grid to the prey. 

The hunter can move in the four cardinal directions, selected using `argmin` over the calculated distances. 


In [ ]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass


# make sure you have the dependencies required here already installed
from torch import nn
import ezkl
import os
import json
import torch

In [ ]:
class NPC(nn.Module):
    def __init__(self):
        super(NPC, self).__init__()

    def forward(self, state):
        # state is a tensor of shape (4,) where the first two elements are the coordinates of the prey and the last two elements are the coordinates of the hunter
        # the prey is at (state[0], state[1]) and the hunter is at (state[2], state[3])
        # (if we move hunter by +1 in x direction what is the distance ?) 
        dist_x_plus = torch.abs(state[:, 0] - (state[:, 2] + 1)) + torch.abs(state[:, 1] - state[:, 3])
        # (if we move hunter by -1 in x direction what is the distance ?)
        dist_x_minus = torch.abs(state[:, 0] - (state[:, 2] - 1)) + torch.abs(state[:, 1] - state[:, 3])
        # (if we move hunter by +1 in y direction what is the distance ?)
        dist_y_plus = torch.abs(state[:, 0] - state[:, 2]) + torch.abs(state[:, 1] - (state[:, 3] + 1))
        # (if we move hunter by -1 in y direction what is the distance ?)
        dist_y_minus = torch.abs(state[:, 0] - state[:, 2]) + torch.abs(state[:, 1] - (state[:, 3] - 1))

        # argmin = 0 -> dist_x_plus
        # argmin = 1 -> dist_x_minus
        # argmin = 2 -> dist_y_plus
        # argmin = 3 -> dist_y_minus
        argmin = torch.argmin(torch.stack([dist_x_plus, dist_x_minus, dist_y_plus, dist_y_minus], dim=0), dim=0)
        return argmin

circuit = NPC()

In [ ]:
gip_run_args = ezkl.PyRunArgs()
gip_run_args.input_visibility = "public"  # matrix and generalized inverse commitments
gip_run_args.output_visibility = "public"   
gip_run_args.param_visibility = "fixed" # should be Tensor(True)
gip_run_args.logrows = 15

In [ ]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [ ]:
# After training, export to onnx (network.onnx) and create a data file (input.json)
shape = [1, 4]

A = torch.randint(size=shape, low=0, high=10, dtype=torch.int64)

# Flips the neural net into inference mode
circuit.eval()

    # Export the model
torch.onnx.export(circuit,               # model being run
                      (A),  # model input (or a tuple for multiple inputs)
                      model_path,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'}},    # variable length axes
                      )

data = dict(
    input_data=A.tolist(),
)

    # Serialize data into file:
json.dump( data, open(data_path, 'w' ))


In [ ]:
circuit.forward(A)

In [ ]:


res = ezkl.gen_settings(model_path, settings_path, py_run_args=gip_run_args)

assert res == True

res = await ezkl.calibrate_settings(data=data_path)


In [ ]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [ ]:
# srs path
res = await ezkl.get_srs( settings_path)

In [ ]:
# now generate the witness file

res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [ ]:

# we pass the witness file to the setup function so as to prepopulate the "fixed" columns of the circuit. 
# in this case we want to force the output to be 0 meaning that the difference between the two matrices is 0
res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        witness_path = witness_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

In [ ]:
# GENERATE A PROOF


proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        "single",
    )

print(res)
assert os.path.isfile(proof_path)

In [ ]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
    )

assert res == True
print("verified")

We can now create and then deploy a vanilla evm verifier. Which can be used to verify the correctness of the hunter's moves ! 

In [ ]:
abi_path = 'test.abi'
sol_code_path = 'test.sol'

res = await ezkl.create_evm_verifier(
        vk_path,
        settings_path,
        sol_code_path,
        abi_path,
    )
assert res == True